In [20]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
arquivo = open("testeab-cd-export.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

In [22]:
for k,v in dados_fire["testeab-cd"].items():
    #print(k,v) # habilite para imprimir as chaves de mais alto nível
    for chave, valor in v.items():
        #print(chave, valor) # habilite para imprimir as chaves inferiores
        pass # desabilite quando reabilitar a linha de cima

In [23]:
lista_dicionarios = []

for k,v in dados_fire["testeab-cd"].items():
    lista_dicionarios.append(v)

        
#print(lista_dicionarios)

In [24]:
dataframe = pd.DataFrame.from_records(lista_dicionarios)

dataframe

,tempo,tipo,user_id,versao
0,1478265613303,inicio,1150166272,A
1,1478622629049,inicio,3410162555,A
2,1478263675557,inicio,1381405382,A
3,1478282921069,inicio,4108159714,A
4,1478272345238,inicio,777092727,B
5,1479319165184,inicio,842512595,B
6,1478268210243,final,1272092754,A
7,1478278982739,inicio,1998226099,B
8,1478263071116,inicio,2423219093,A
9,1478263078153,inicio,2423219093,A


In [25]:
agrupado = dataframe.groupby(by=["user_id", "versao"])

users = set(dataframe.user_id) # We convert it to a set so that each user appear only once
users = list(users) 

In [26]:
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="final"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="final"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                if delta_t < 300000:
                    resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})
                

In [27]:
dataframe_resumo = pd.DataFrame.from_records(resultados)

In [28]:
dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

In [29]:
dataframe_resumo

,delta_t,user_id,versao
0,59.713,285365760,B
1,41.444,3789181448,A
2,18.755,528519713,A
3,24.113,2830537766,A
4,26.782,2409261628,B
5,7.846,3185562697,B
6,10.745,1272092754,A
7,155.030,1243295326,B
8,218.525,1591238248,B
9,80.565,2203327087,A


In [30]:
dataframe_resumo.to_csv('dados_preparados.csv')

print('Comando executado')

Comando executado


In [31]:
a=0
tempoa=0

b=0
tempob=0


for i in range(len(dataframe_resumo)):
    versao = dataframe_resumo.iloc[i]['versao']
    if versao == "A":
        a+=1
        tempoa+=dataframe_resumo.iloc[i]['delta_t']
    elif versao == "B":
        b+=1
        tempob+=dataframe_resumo.iloc[i]['delta_t']
else: 
    pass
    
media_a = tempoa/a
media_b = tempob/b


print("N˚ A:",a)
print("N˚ B:",b)
print()
print("Total A:",tempoa)
print("Total B:",tempob)
print()
print("Média A:",media_a)
print("Média B:",media_b)

N˚ A: 29
N˚ B: 28

Total A: 1726.261
Total B: 1597.781

Média A: 59.5262413793
Média B: 57.0636071429
